In [1]:
#effigy sync process

import os
from datetime import datetime
from PIL import Image 
import glob
import ntpath

import mysql.connector
from mysql.connector import errorcode
import logging


logging.basicConfig(level=logging.DEBUG, filename='effigySync.log', format='%(asctime)s %(name)s %(levelname)s:%(message)s')
logger = logging.getLogger(__name__)


In [2]:

#SOURCEPATH='Y:/server/EFFIGY_PICS'
SOURCEPATH='Y:/server/EFFIGY_PICS2'
#SOURCEPATH = 'C:/Users/user/Pictures'


In [3]:
def fixSlashes(val):
    val = val.replace('\\', '/')
    return val   


In [4]:
def getDt():
    now = datetime.now()
    dt = now.strftime('%Y-%m-%d %H:%M:%S')
    return(dt)


In [5]:
getDt()

'2020-04-15 18:52:38'

In [6]:

a=set()
#folders = []
# r=root, d=directories, f = files
for r, d, f in os.walk(SOURCEPATH):       
    for f1 in f:
        ext = os.path.splitext(f1)[1]
        a.add(ext) 

In [7]:
def createThumbnailss(srcPath,destPath,size=120):
    for infile in glob.glob(srcPath):
        print("infile",infile)
        file, ext = os.path.splitext(infile)
        print(file,ext)
        print(destPath)
        im = Image.open(infile)
        im.thumbnail(size,size)
        im.save(destPath + "/thumbnail1.png", "PNG")

In [8]:
def createThumbnails(srcFile, destPath, basewidth = 30):
    img = Image.open(srcFile)
    file = ntpath.basename(srcFile)

    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    destFile =  "./"+ file
    img.save(destFile)
    

In [9]:
createThumbnails("Y:\server\EFFIGY_PICS2\Zdjecia\other/IMG_0132.jpg","c:/temp",)

In [10]:
def getID(tbl,thisid, key,val):
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select " + thisid + " from " +  tbl  + " where " + key + "= %s"
        #print (ssql)
        mycursor.execute(ssql, (val,))
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        newid = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            newid.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:" + val
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed"+ val)
        # obejct is a tuple we retrieve only id
        print(newid)
        if len(newid) >= 1:
            id = newid[0][0]
        else:
            id = -1
        return (id)

In [11]:
def getImageID(tbl,thisid, key,val,key2,val2):
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select " + thisid + " from " +  tbl  + " where " + key + "= %s and  " + key2 + "= %s"
        #print (ssql)
        mycursor.execute(ssql, (fixSlashes(val),val2,))
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        newid = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            newid.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:" + val
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed"+ val)
        # obejct is a tuple we retrieve only id
        print(newid)
        if len(newid) >= 1:
            id = newid[0][0]
        else:
            id = -1
        return (id)

In [12]:
newid= getImageID("media","media_id", "path","c:\\","name","test.jpg")

[]


In [13]:
def getValidExtensions():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select extension from supported_ext order by 1"
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        # converting it to a SET, fast lookup
        ext_set=set()  
        for i in rec:
            ext_set.add (i[0])
        return (ext_set)

In [14]:
def getSkipFolder():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select skip_root from skip_root"
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        # converting it to a SET, fast lookup
        _set=set()  
        for i in rec:
            _set.add (i[0])
        return (_set)

In [15]:
getSkipFolder()

{'Y:\\server\\EFFIGY_PICS2\\Zdjecia\\tomcat-ZDJECIA',
 'Y:\\server\\EFFIGY_PICS\\Zdjecia\\tomcat-ZDJECIA'}

In [16]:
import functools 
import operator  
def convertTuple(tup): 
    str1 =  ''.join(str(tup)) 
    return str1

In [17]:
#get valid extensions
EXT = getValidExtensions()
#  check
'avi' in EXT
EXT

{'.avi', '.jpeg', '.jpg', '.mov', '.mp3', '.mp4', '.mpg', '.png', '.wmv'}

In [18]:
def insertRoot (root):
    logger.info("Starting Request")
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into loc_root values (null, %s)"
        root = fixSlashes(root)
        #print (root)
        mycursor.execute(ssql, (root,))
        mydb.commit()
        #get newly created ID
        newid= getID("loc_root","root_id", "root",fixSlashes(root))
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Root-Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Root - DB Closed")
        return (newid)

In [19]:
getID("loc_folder","folder_id", "folder_path","C:ggg/Uffsssehhffrsggs/user/Pictures/2017-11111111111")

[]


-1

In [20]:
def insertFolder (root_id,root,folder,folderPath):
    logger.info("Starting Request")
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into loc_folder values (null,%s, %s,%s,%s)"
        root = fixSlashes(root)
        folder = fixSlashes(folder)
        folderPath = fixSlashes(folderPath)
        #print (ssql)
        mycursor.execute(ssql, (root_id,root,folder,folderPath,))
        mydb.commit()
        #get newly created ID
        newid= getID("loc_folder","folder_id", "folder_path",fixSlashes(folderPath))
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        return (newid)

In [21]:
def skipRoot(root):
    skip=False
    toSkip="Y:\server\EFFIGY_PICS\Zdjecia\\tomcat-ZDJECIA"
    if len(root) >= len(toSkip):
        if (fixSlashes(toSkip).upper() == fixSlashes(root[:len(toSkip)]).upper())  :
            skip=True
    #else:
    #    print("not matching length")
    return(skip)

In [22]:
def skipFolder(root):
    skip=False
    toSkip=getSkipFolder()
    for folder in toSkip:
        if len(root) >= len(folder):
            print(folder)
            print(fixSlashes(folder).upper())
            print(fixSlashes(root[:len(folder)]).upper()) 
            if (fixSlashes(folder).upper() == fixSlashes(root[:len(folder)]).upper())  :
                skip=True
                return(skip)
    return(skip)
        


In [23]:
skipFolder("Y:\server\EFFIGY_PICS2/Zdjecia\\tomcat-ZDJECIA")

Y:\server\EFFIGY_PICS\Zdjecia\tomcat-ZDJECIA
Y:/SERVER/EFFIGY_PICS/ZDJECIA/TOMCAT-ZDJECIA
Y:/SERVER/EFFIGY_PICS2/ZDJECIA/TOMCAT-ZDJECI
Y:\server\EFFIGY_PICS2\Zdjecia\tomcat-ZDJECIA
Y:/SERVER/EFFIGY_PICS2/ZDJECIA/TOMCAT-ZDJECIA
Y:/SERVER/EFFIGY_PICS2/ZDJECIA/TOMCAT-ZDJECIA


True

In [24]:
#newid = insertFolder(1, "C:/Users/user/Pictures","2017-1111", "C:ggg/Uffsssehhffrsggs/user/Pictures/2017-1221111111111")
#newid

In [25]:
def inserImage (key,file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext):
    logger.info("Saving image")
    print(key,file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext)
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into media values (null,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        path = fixSlashes(path)
        parent_folder = fixSlashes(parent_folder)
        #print (ssql)
        mycursor.execute(ssql, ( key,file_name,media_type,folder_id,path,parent_folder_id,parent_folder,"sDesc",file_name,0,ext,getDt(),getDt()))
        mydb.commit()
        #get newly created ID
        newid= getImageID("media","media_id", "path",path,"name",file_name)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        return (newid)

In [26]:
def importFoldersAndImages ():
    #get Valid extensions first
    c=0
    EXT = getValidExtensions()
    #  check
    '' in EXT
    print(EXT)
    for r, d, f in os.walk(SOURCEPATH):
        #print("dir",d,r)
        if len(d) >0:
            if  skipRoot(r):
                print("Skipped Root:",r)
            else:
                idR = insertRoot(r)
                #print("Processed Root:",idR,r)
                for dir in d:
                    #print ("will INSERT dir:",idR,dir)
                    #print("about to insert", idR,r,dir,r + '/' + dir)
                    idF = insertFolder(idR,r,dir,r + "/"+ dir)
                    #print("Created folder",idF)
                    pth = r +"/"+dir
                    #print("path", pth)
                    files = [f for f in os.listdir(pth) if os.path.isfile(os.path.join(pth,f))]
                    for f in files:
                        file_ext= os.path.splitext(f)[1].lower()
                        if (file_ext in EXT):
                            #print("Processing:" + f)
                            key=r +"/"+dir + "/" + f
                            #inserImage(file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext):                       
                            idM = inserImage(key,f,'P',idF, r +"/"+dir, idR,r,"",file_ext)
                            #print(idM)
                           
    return(c)                    
                    
    
                    
                    
                
        
    

In [27]:
c= importFoldersAndImages()
print ("total:",c)

{'.avi', '.mp4', '.jpeg', '.mov', '.jpg', '.mpg', '.mp3', '.png', '.wmv'}
[(16,)]
[(21,)]
Y:/server/EFFIGY_PICS2/Zdjecia/20041008 001.JPG 20041008 001.JPG P 21 Y:/server/EFFIGY_PICS2/Zdjecia 16 Y:/server/EFFIGY_PICS2  .jpg
[(31,)]
Y:/server/EFFIGY_PICS2/Zdjecia/20041008 002.jpg 20041008 002.jpg P 21 Y:/server/EFFIGY_PICS2/Zdjecia 16 Y:/server/EFFIGY_PICS2  .jpg
[(32,)]
[(17,)]
[(22,)]
Y:/server/EFFIGY_PICS2\Zdjecia/other/IMG_0025.jpg IMG_0025.jpg P 22 Y:/server/EFFIGY_PICS2\Zdjecia/other 17 Y:/server/EFFIGY_PICS2\Zdjecia  .jpg
[(33,)]
Y:/server/EFFIGY_PICS2\Zdjecia/other/IMG_0028.jpg IMG_0028.jpg P 22 Y:/server/EFFIGY_PICS2\Zdjecia/other 17 Y:/server/EFFIGY_PICS2\Zdjecia  .jpg
[(34,)]
Y:/server/EFFIGY_PICS2\Zdjecia/other/IMG_0131.jpg IMG_0131.jpg P 22 Y:/server/EFFIGY_PICS2\Zdjecia/other 17 Y:/server/EFFIGY_PICS2\Zdjecia  .jpg
[(35,)]
Y:/server/EFFIGY_PICS2\Zdjecia/other/IMG_0132.jpg IMG_0132.jpg P 22 Y:/server/EFFIGY_PICS2\Zdjecia/other 17 Y:/server/EFFIGY_PICS2\Zdjecia  .jpg
[(36,)]


In [28]:
def getImages():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select media_id,path, name, media_key,media_type_id from media"
        #print (ssql)
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        _set=set()  
        for i in rec:
            _set.add (i)
        return (_set)

In [34]:
def getImageExifAttributes(pth):
    exif= {}
    ex=""
    # Read image 
    img = Image.open(pth) 
    dic = img._getexif()
    exif["WIDTH"] =  str(dic.get(40963)) 
    exif["HEIGHT"] = str( dic.get(40962))
    exif["CAMERA"] = str( dic.get(271))
    exif["MODEL"] =  str(dic.get(272))
    exif["DTTAKEN"] = str( dic.get(36868))
    exif["DTMODIFY"] = str( dic.get(36867))
    exif["FSTOP"] =  str(dic.get(42036))
    exif["EXPOSURE"] =  str(dic.get(40960))
    exif["ORIENTATION"] = str( dic.get(274))
    exif["FLASH"] =  str(dic.get(7385))
    exif["SHUTTER"] =  str(dic.get(37377))
    exif["APERTURE"] = str( dic.get(37378))
    exif["ISOSPEED"] = str( dic.get(34855))
    exif["DIGIZOOM"] =  str(dic.get(41988))
    print("ORIENTATION",exif["ORIENTATION"]) 
    for x in dic:
         if x != 37500 and x != 37510:
            ex= ex + (str(x) +  ":" +  str(dic.get(x)) + chr(13) +chr(10) )  
    exif["EXIF"] =  str(ex) 
    return(exif)

In [35]:
getImageExifAttributes("Y:/server/EFFIGY_PICS2/Zdjecia/other/IMG_0131.jpg")

ORIENTATION None


{'WIDTH': '2592',
 'HEIGHT': '1944',
 'CAMERA': 'Canon',
 'MODEL': 'Canon PowerShot A450',
 'DTTAKEN': '2007:12:01 21:45:04',
 'DTMODIFY': '2007:12:01 21:45:04',
 'FSTOP': 'None',
 'EXPOSURE': "b'0100'",
 'ORIENTATION': 'None',
 'FLASH': 'None',
 'SHUTTER': '(189, 32)',
 'APERTURE': '(95, 32)',
 'ISOSPEED': '200',
 'DIGIZOOM': '(2592, 2592)',
 'EXIF': "36864:b'0220'\r\n37121:b'\\x01\\x02\\x03\\x00'\r\n37122:(5, 1)\r\n36867:2007:12:01 21:45:04\r\n36868:2007:12:01 21:45:04\r\n37377:(189, 32)\r\n37378:(95, 32)\r\n37380:(0, 3)\r\n37381:(95, 32)\r\n37383:5\r\n37385:89\r\n37386:(5400, 1000)\r\n40961:1\r\n40962:1944\r\n41486:(2592000, 194)\r\n41487:(1944000, 145)\r\n40963:2592\r\n41488:2\r\n271:Canon\r\n272:Canon PowerShot A450\r\n531:1\r\n41495:2\r\n41728:b'\\x03'\r\n282:(180, 1)\r\n283:(180, 1)\r\n41985:0\r\n34855:200\r\n296:2\r\n41986:0\r\n40960:b'0100'\r\n41987:0\r\n306:2007:12:01 21:45:04\r\n41988:(2592, 2592)\r\n41990:0\r\n34665:266\r\n4097:2592\r\n4098:1944\r\n33434:(1, 60)\r\n33437:(2

In [31]:
def insertMediaAttributes (width, height, camera, model, dttaken, dtmodify,\
                          fstop, exposure, orientation, flash, shutter, aperture,\
                          isospeed,  digizoom,  exif,  media_id ,media_key):
    logger.info("Saving Attributes" + media_key)
    #print(key,file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext)
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Passw0rd',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into attrib values (null,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        #print (ssql)
        mycursor.execute(ssql, ( width, height, camera, model, dttaken, dtmodify,\
                          fstop,  exposure, orientation, flash, shutter, aperture,\
                          isospeed, digizoom, exif, media_id, media_key, getDt()))
        mydb.commit()
        #get newly created ID
        newid= getID("attrib","atrib_id", "media_id",media_id)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        return (newid)

In [32]:
def ProcessAttributes():
    img_set = getImages()
    for media_id,path,fname, media_key, media_type_id in img_set:
        pth = fixSlashes(path +"/"+fname)
        print(pth)
        if media_type_id == 'P':
            exif=getImageExifAttributes(pth)
            idA = insertMediaAttributes(exif["WIDTH"],exif["HEIGHT"],exif["CAMERA"],exif["MODEL"],exif["DTTAKEN"],exif["DTMODIFY"],exif["FSTOP"],exif["EXPOSURE"],exif["ORIENTATION"],exif["FLASH"],exif["SHUTTER"],exif["APERTURE"],exif["ISOSPEED"],exif["DIGIZOOM"],exif["EXIF"], media_id ,media_key)
            #atrib_id,  width, height,  camera, model,  dttaken,  dtmodify,  fstop,  exposure,  orientation,  flash,  shutter,  aperture,  isospeed,  digizoom,  exif,  media_id ,media_key,'


In [33]:
ProcessAttributes()

Y:/server/EFFIGY_PICS2/Zdjecia/other/IMG_0132.jpg
ORIENTATION 1
569


AttributeError: 'TypeError' object has no attribute 'errno'

In [ ]:

img_set = getImages()
for media_id,path,fname, media_key, media_type_id in img_set:
    pth = fixSlashes(path +"/"+fname)
    print(pth)
    if media_type_id == 'P':
        exif=getImageExifAttributes(pth)
        print (len(exif["EXIF"]))

    

In [ ]:
exif = getImageExifAttributes("Y:/server/EFFIGY_PICS2\\Zdjecia/other/IMG_0025.jpg")
exif["WIDTH"]
#print(exif)

#for e in exif:
#    print(e)
print(str(exif["APERTURE"]))
print(exif["APERTURE"])
print(convertTuple(exif["APERTURE"]))
#idA = inserMediaAttributes(exif["WIDTH"],exif["HEIGHT"],exif["CAMERA"],exif["MODEL"],exif["DTTAKEN"],exif["DTMODIFY"],exif["FSTOP"],exif["EXPOSURE"],exif["ORIENTATION"],exif["FLASH"],exif["SHUTTER"],exif["APERTURE"],exif["ISOSPEED"],exif["DIGIZOOM"],"","1" ,"1")
             

In [ ]:
path = "C:/GIT_REPOSITORY/REPO/effigy/imageLib/test.jpg"
from PIL import Image
# Read image 
img = Image.open(path) 
  
# Output Images 
img.show() 

# prints format of image 
print(img.format) 

# prints mode of image 
print(img.mode) 

In [ ]:

#path = "C:/GIT_REPOSITORY/REPO/effigy/imageLib/geotag.jpg"
path= "Y:/server/EFFIGY_PICS2/Zdjecia/other/IMG_0131.jpg"
# Python program to read 
# image using PIL module 
  
# importing PIL 
from PIL import Image 
  
# Read image 
img = Image.open(path) 
  
# Output Images 
img.show() 
  
# prints format of image 
print(img.format) 
  
# prints mode of image 
print(img.mode) 
dic = img._getexif()
width = dic.get(40963)
height = dic.get(40962)
camera = dic.get(271)
model = dic.get(272)
dttaken = dic.get(36868)
dtmod= dic.get(36867)
fstop= dic.get(42036)
exposure = dic.get(40960) 
orientation=dic.get(274)
flash = dic.get(7385)
shutter = dic.get(37377)
apperture=dic.get(37378) 
isospeed=dic.get(34855) 
digzoom=dic.get(41988)
exif=""
